 ### Smiles Retrieval Using PubChem API


In [1]:
import pandas as pd
import pubchempy as pcp

from tqdm import tqdm
import numpy as np

##### Load DrugComb Dataset

In [2]:
drug_comb=pd.read_csv("drug_interaction_drugcomb.csv",low_memory=False)

In [3]:
drug_comb.head()

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S
0,1,5-FU,ABT-888,A2058,uM,uM,30.869,3.865915,6.256584,-2.951386,5.536903,5.126836,3.267734,11.471,-0.441,22.545,39.193,19.839
1,2,5-FU,ABT-888,A2058,uM,uM,27.460,8.247403,12.333896,3.125927,11.614215,5.126836,3.267734,11.471,-0.441,24.135,30.785,16.430
2,3,5-FU,ABT-888,A2058,uM,uM,29.901,6.063440,11.660209,2.452239,10.940528,5.126836,3.267734,11.471,-0.441,25.561,34.241,18.871
3,4,5-FU,ABT-888,A2058,uM,uM,24.016,-4.280231,5.145209,-4.062761,4.425528,5.126836,3.267734,11.471,-0.441,16.661,31.371,12.986
4,5,5-FU,AZD1775,A2058,uM,uM,66.847,12.284698,15.765467,10.409407,18.656340,5.126836,0.266027,11.471,25.164,76.501,57.193,30.212


In [4]:
drug_comb.shape

(1048575, 18)

In [5]:
drug_comb['drug_row'].isna().sum()  #Counts the number of missing (NaN) values in the 'drug_row' column 

np.int64(0)

In [6]:
drug_comb['drug_col'].isna().sum()  #Counts the number of missing (NaN) values in the 'drug_col' column 

np.int64(582542)

##### Missing drug names in the drug_col column were not imputed, as such data is considered sensitive and unsuitable for imputation. Instead, rows containing missing values in drug_col were removed to ensure the integrity of downstream processing.

In [7]:
drug_comb = drug_comb.dropna(subset='drug_col')  # Drops all rows from the DataFrame where the 'drug_col' entry is missing (NaN)

In [8]:
drug_comb['drug_col'].isna().sum()

np.int64(0)

In [9]:
a=len(np.unique(np.concatenate([drug_comb.drug_row.unique(),drug_comb.drug_col.unique()])))
print(f"total number of drug involved in interaciton dataset {a}")

total number of drug involved in interaciton dataset 4150


In [10]:
drug_comb.shape

(466033, 18)

In [11]:
drug_comb[drug_comb['drug_row'].isna() | drug_comb['drug_col'].isna()] # Selects all rows from drug_comb where either 'drug_row' or 'drug_col' has missing (NaN) values

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S


##### The distinct drug names are compiled into a new DataFrame called drug_df to facilitate further analysis, such as retrieving SMILES strings

In [12]:
unique_drugs = pd.unique(drug_comb[['drug_row', 'drug_col']].values.ravel())
drug_df = pd.DataFrame(unique_drugs, columns=['drug'])
drug_df.head()

,drug
0,5-FU
1,ABT-888
2,AZD1775
3,BEZ-235
4,BORTEZOMIB


In [13]:
drug_df.shape

(4150, 1)

##### Fetching SMILES Strings from PubChem by Drug Name

In [14]:
import requests

failed_names = []

def get_smiles_from_name(name):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/property/IsomericSMILES/TXT'
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text.strip()
    except Exception as e:
        print(f"Error retrieving SMILES for {name}: {e}")
        failed_names.append(name)
        return None


##### Adding a new 'smiles' column to drug_df by applying the get_smiles_from_name function to each drug name, fetching its SMILES string from PubChem

In [21]:
# Enabling progress_apply to show progress bars with pandas apply
tqdm.pandas()
drug_df['smiles'] = drug_df['drug'].progress_apply(get_smiles_from_name)

  2%|█▌                                                                              | 81/4150 [01:13<58:54,  1.15it/s]

Error retrieving SMILES for (+/-)-SULFINPYRAZONE: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(+/-)-SULFINPYRAZONE/property/IsomericSMILES/TXT


 10%|███████▌                                                                       | 395/4150 [06:03<55:53,  1.12it/s]

Error retrieving SMILES for (6-)E-<U+200B>AMINOCAPROIC ACID: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(6-)E-%3CU+200B%3EAMINOCAPROIC%20ACID/property/IsomericSMILES/TXT


 14%|███████████                                                                    | 584/4150 [08:56<54:35,  1.09it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████▌| 4129/4150 [1:02:29<00:18,  1.15it/s]

Error retrieving SMILES for AC1Q4OOX: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/AC1Q4OOX/property/IsomericSMILES/TXT


100%|███████████████████████████████████████████████████████████████████████████▊| 4142/4150 [1:02:40<00:07,  1.07it/s]

Error retrieving SMILES for Vincristine Sulfate, Apocynaceae sp.: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Vincristine%20Sulfate,%20Apocynaceae%20sp./property/IsomericSMILES/TXT


100%|████████████████████████████████████████████████████████████████████████████| 4150/4150 [1:02:49<00:00,  1.10it/s]


##### Save the drug_df DataFrame with drug names and their SMILES strings to a CSV file


In [22]:
drug_df.to_csv("final_drug_with_smiles.csv")

In [14]:
drug_df=pd.read_csv("final_drug_with_smiles.csv",index_col=0)

In [15]:
drug_df.isnull().sum()

drug        0
smiles    106
dtype: int64

In [16]:
import requests
from bs4 import BeautifulSoup
import time

failed_names = []

def get_from_pubchem(name):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/property/IsomericSMILES/TXT'
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text.strip()
    except Exception as e:
        print(f"PubChem failed for {name}: {e}")
        return None

def get_from_zinc(name):
    try:
        search_url = f"https://zinc15.docking.org/substances/search/?q={name}"
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(search_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        smiles_div = soup.find("div", class_="smiles")
        if smiles_div:
            return smiles_div.text.strip()
        return None
    except Exception as e:
        print(f"ZINC15 failed for {name}: {e}")
        return None

def get_from_pubchem_cid(name):
    try:
        cid_url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/cids/TXT'
        response = requests.get(cid_url, timeout=10)
        response.raise_for_status()
        cid = response.text.strip().split('\n')[0]
        prop_url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/IsomericSMILES/TXT'
        prop_response = requests.get(prop_url, timeout=10)
        prop_response.raise_for_status()
        return prop_response.text.strip()
    except Exception as e:
        print(f"PubChem-CID failed for {name}: {e}")
        return None

def get_from_nci(name):
    try:
        url = f"https://cactus.nci.nih.gov/chemical/structure/{name}/smiles"
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.text.strip()
        return None
    except Exception as e:
        print(f"NCI Resolver failed for {name}: {e}")
        return None

def get_from_cir(name):
    try:
        url = f"https://cactus.nci.nih.gov/chemical/structure/{name}/smiles"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text.strip()
    except Exception as e:
        print(f"CIR failed for {name}: {e}")
        return None

def error_msg(name):
    print(f"No online match found for {name}, checking local DB...")  
    return None


def get_from_chembl(name):
    try:
        url = f'https://www.ebi.ac.uk/chembl/api/data/molecule?pref_name={name}&format=json'
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        if data["page_meta"]["total_count"] > 0:
            molecule = data["molecules"][0]
            if "molecule_structures" in molecule and molecule["molecule_structures"]:
                return molecule["molecule_structures"].get("canonical_smiles")
        return None
    except Exception as e:
        print(f"ChEMBL failed for {name}: {e}")
        return None

# Master pipeline
def get_smiles_from_name(name):
    methods = [
        get_from_pubchem,
        get_from_pubchem_cid,
        get_from_zinc,
        get_from_nci,
        get_from_cir,
        get_from_chembl,
    ]
    
    for method in methods:
        result = method(name)
        if result:
            return result
        else:
            print(f" {method.__name__} could not find SMILES for '{name}'")
        time.sleep(1)  # To be gentle on APIs

    print(f" None of the APIs could find SMILES for '{name}'\n")
    failed_names.append(name)
    return None

tqdm.pandas()
drug_df['smiles'] = drug_df['drug'].progress_apply(get_smiles_from_name)


  2%|█▌                                                                            | 80/4150 [01:10<1:01:10,  1.11it/s]

PubChem failed for (+/-)-SULFINPYRAZONE: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(+/-)-SULFINPYRAZONE/property/IsomericSMILES/TXT
 get_from_pubchem could not find SMILES for '(+/-)-SULFINPYRAZONE'
PubChem-CID failed for (+/-)-SULFINPYRAZONE: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(+/-)-SULFINPYRAZONE/cids/TXT
 get_from_pubchem_cid could not find SMILES for '(+/-)-SULFINPYRAZONE'
 get_from_zinc could not find SMILES for '(+/-)-SULFINPYRAZONE'


  9%|███████▌                                                                       | 394/4150 [06:01<54:47,  1.14it/s]

PubChem failed for (6-)E-<U+200B>AMINOCAPROIC ACID: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(6-)E-%3CU+200B%3EAMINOCAPROIC%20ACID/property/IsomericSMILES/TXT
 get_from_pubchem could not find SMILES for '(6-)E-<U+200B>AMINOCAPROIC ACID'
PubChem-CID failed for (6-)E-<U+200B>AMINOCAPROIC ACID: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/(6-)E-%3CU+200B%3EAMINOCAPROIC%20ACID/cids/TXT
 get_from_pubchem_cid could not find SMILES for '(6-)E-<U+200B>AMINOCAPROIC ACID'
 get_from_zinc could not find SMILES for '(6-)E-<U+200B>AMINOCAPROIC ACID'
 get_from_nci could not find SMILES for '(6-)E-<U+200B>AMINOCAPROIC ACID'
CIR failed for (6-)E-<U+200B>AMINOCAPROIC ACID: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/(6-)E-%3CU+200B%3EAMINOCAPROIC%20ACID/smiles
 get_from_cir could not find SMILES for '(6-)E-<U+200B>AMINOCAPROIC ACID'
 get_fro

 10%|███████▎                                                                     | 395/4150 [06:15<4:55:06,  4.72s/it]

...

100%|███████████████████████████████████████████████████████████████████████████▊| 4141/4150 [1:23:19<00:08,  1.04it/s]

PubChem failed for Vincristine Sulfate, Apocynaceae sp.: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Vincristine%20Sulfate,%20Apocynaceae%20sp./property/IsomericSMILES/TXT
 get_from_pubchem could not find SMILES for 'Vincristine Sulfate, Apocynaceae sp.'
PubChem-CID failed for Vincristine Sulfate, Apocynaceae sp.: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Vincristine%20Sulfate,%20Apocynaceae%20sp./cids/TXT
 get_from_pubchem_cid could not find SMILES for 'Vincristine Sulfate, Apocynaceae sp.'
 get_from_zinc could not find SMILES for 'Vincristine Sulfate, Apocynaceae sp.'
 get_from_nci could not find SMILES for 'Vincristine Sulfate, Apocynaceae sp.'
CIR failed for Vincristine Sulfate, Apocynaceae sp.: 500 Server Error: INTERNAL SERVER ERROR for url: https://cactus.nci.nih.gov/chemical/structure/Vincristine%20Sulfate,%20Apocynaceae%20sp./smiles
 get_from_cir could not find SMI

100%|███████████████████████████████████████████████████████████████████████████▊| 4142/4150 [1:23:34<00:40,  5.11s/it]

 None of the APIs could find SMILES for 'Vincristine Sulfate, Apocynaceae sp.'



100%|████████████████████████████████████████████████████████████████████████████| 4150/4150 [1:23:43<00:00,  1.21s/it]


In [17]:
drug_df.head()

,drug,smiles
0,5-FU,C1=C(C(=O)NC(=O)N1)F
1,ABT-888,C[C@@]1(CCCN1)C2=NC3=C(C=CC=C3N2)C(=O)N
2,AZD1775,CC(C)(C1=NC(=CC=C1)N2C3=NC(=NC=C3C(=O)N2CC=C)N...
3,BEZ-235,CC(C)(C#N)C1=CC=C(C=C1)N2C3=C4C=C(C=CC4=NC=C3N...
4,BORTEZOMIB,B([C@H](CC(C)C)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)...


In [18]:
drug_df.isnull().sum()

drug       0
smiles    98
dtype: int64

In [19]:
drug_df.to_csv("drug_to_smiles_after_master_pipeline.csv")

In [2]:
drug_df=pd.read_csv("drug_to_smiles_after_master_pipeline.csv")

In [21]:
len(failed_names)

98

In [22]:
failed_names

['(6-)E-<U+200B>AMINOCAPROIC ACID',
 'GYNOSTEMMA EXTRACT',
 'METHYL-HESPERIDIN',
 'PIFITHRIN-A (PFTA)',
 'TETRAETHYLENEPENTAMINE 5HCL',
 'Triciribinephosphate',
 'ONO-4059 analog',
 "3',4'-Diacetylafzelin",
 'ERK 11e',
 'AGN-PC-0MU5N5',
 'ZINC18182079',
 'AK176003',
 'ZINC21999985',
 'DSSTox_CID_31408',
 'ZINC84688828',
 'QCR-20',
 'AC1NPCOI',
 'Pd-1/pd-l1 inhibitor 1',
 'MOLPORT-042-665-727',
 'ZINC242723022',
 'AC1OF4NF',
 'Ergosterine',
 'AC1O7H3F',
 'ZINC250325905',
 'AC1NUNML',
 'AK198640',
 'AC1LAYFC',
 'AC1LU7HW',
 'ZD7288 HYDRATE',
 'AC1O53DL',
 'Bacitracin (Zinc)',
 'TIC10 isomer',
 'AK170455',
 'AK-77283',
 'AC1L1KJK',
 'ZINC208949883',
 'ZINC11616871',
 'ZINC117147304',
 'KS-00001CUZ',
 'CTK8E8456',
 'CTK8F0346',
 'ZINC208012538',
 'ZINC17545571',
 '04/04/7218',
 'Curcumin, Curcuma longa L.',
 'ZINC34894448',
 'AC1L9B3J',
 'ZINC100015731',
 'AC1L2QK3',
 'KS-00001D5I',
 'AC1Q29DP',
 'KS-00000Y5E',
 'KGBPLKOPSFDBOX-UHFFFAOYSA-N',
 'AC1L1IJY',
 'AK160201',
 'AK174336',
 'AC1L1D

In [3]:
drug_df[drug_df['drug']=='GYNOSTEMMA EXTRACT']

,Unnamed: 0,drug,smiles
1106,1106,GYNOSTEMMA EXTRACT,NaN


In [6]:
drug_df_without_smiles=drug_df[drug_df['smiles'].isna()]

In [7]:
drug_df_without_smiles   #missing smiles 

,Unnamed: 0,drug,smiles
393,393,(6-)E-<U+200B>AMINOCAPROIC ACID,NaN
1106,1106,GYNOSTEMMA EXTRACT,NaN
1313,1313,METHYL-HESPERIDIN,NaN
1548,1548,PIFITHRIN-A (PFTA),NaN
1852,1852,TETRAETHYLENEPENTAMINE 5HCL,NaN
...,...,...,...
4101,4101,AC1L34YQ,NaN
4102,4102,AC1M4BZH,NaN
4124,4124,AC1L1EUK,NaN
4127,4127,AC1Q4OOX,NaN
